<a href="https://colab.research.google.com/github/jish1398/Nlu_Intent/blob/master/nlu_intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[spacy];
!python -m spacy download en;


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [10]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:restaurant_search
- i'm looking for a place to eat
- I want to grab lunch
- I am searching for a dinner spot
- i'm looking for a place in the [north](location) of town
- show me [chinese](cuisine) restaurants
- show me a [mexican](cuisine) place in the [centre](location)
- i am looking for an [indian](cuisine) spot
- search for restaurants
- anywhere in the [west](location)
- anywhere near [18328](location)
- I am looking for [asian fusion](cuisine) food
- I am looking a restaurant in [29432](location)

## intent:thankyou
- thanks!
- thank you
- thx
- thanks very much

"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [11]:
nlu_config = """
language: en
pipeline: 
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
  intent_tokenization_flag: true
  intent_split_symbol: "+"
"""
%store nlu_config > nlu_config.yml


Writing 'nlu_config' (str) to file 'nlu_config.yml'.


In [12]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose

2018-10-30 09:57:27 INFO     rasa_nlu.training_data.loading  - Training data format of nlu.md is md
2018-10-30 09:57:27 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 22 (3 distinct intents)
	- Found intents: 'greet', 'thankyou', 'restaurant_search'
	- entity examples: 8 (2 distinct entities)
	- found entities: 'location', 'cuisine'

2018-10-30 09:57:27 INFO     rasa_nlu.model  - Starting to train component intent_featurizer_count_vectors
2018-10-30 09:57:28 INFO     rasa_nlu.model  - Finished training component.
2018-10-30 09:57:28 INFO     rasa_nlu.model  - Starting to train component intent_classifier_tensorflow_embedding
2018-10-30 09:57:32.195966: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-10-30 09:57:32 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Accuracy is updated every 10 epochs
Epochs: 100% 300/300 [00:01<00:0

In [13]:
from rasa_nlu.model import Interpreter
import json
interpreter = Interpreter.load("./models/current/nlu")
message = "let's see some italian restaurants"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

INFO:tensorflow:Restoring parameters from ./models/current/nlu/intent_classifier_tensorflow_embedding.ckpt


{
  "intent": {
    "name": "restaurant_search",
    "confidence": 0.830917239189148
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "restaurant_search",
      "confidence": 0.830917239189148
    },
    {
      "name": "thankyou",
      "confidence": 0.2644999623298645
    },
    {
      "name": "greet",
      "confidence": 0.1818365901708603
    }
  ],
  "text": "let's see some italian restaurants"
}
